# The basics of querying CMR
[Reference](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html)

The simplest query to CMR is a collection-level search with no filter constraints:

    GET https://cmr.earthdata.nasa.gov/search/collections

We will use this query to demonstrate the basic functionality of CMR search.
What can we do with this query?

In [ ]:
import requests
import xml.dom.minidom

response = requests.get("https://cmr.earthdata.nasa.gov/search/collections")

The default response is an xml document of collection result references. Let's pretty print the output.

In [ ]:
# The default response (xml reference) is an xml document of result references. Let's pretty print the output
response_as_dom = xml.dom.minidom.parseString(response.text)
xml_reference_response = response_as_dom.toprettyxml()
print(xml_reference_response)

## Parsing the xml results
We can extract information from the results in a programatic way using xpath

How many collection results are there?

In [ ]:
import xml.etree.ElementTree as et
doc = et.fromstring(response.text)

print("Total number of collection results: " + doc.findtext('hits'))   

How many collection results were returned?

In [ ]:
references = doc.findall('references/reference')
print("No. of results returned: " + str(len(references)))  

Where is the first collection result?

In [ ]:
print("First result reference: " + str(references[0].find('location').text))

What is the human-readable name of the first collection result?

In [ ]:
print("First result name: " + str(references[0].find('name').text))

What is the unique ID (concept-id) of the first collection result?

In [ ]:
print("First result concept id: " + str(references[0].find('id').text))

## Going further than the first 10 results
CMR search supports paging parameters to iterate through 'pages' of results.
You can select your page size and your page number. Page size can be a positive integer between 1 and 2000. Page number can be a positive integer.
Note: for harvesting use cases we do not recommend paging through results. See: [scrolling](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#scrolling-details)

Let's try getting 20 results instead of the default value of 10.

    GET https://cmr.earthdata.nasa.gov/search/collections?page_size=20

In [ ]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?page_size=20")
doc = et.fromstring(response.text)
print("No. of results returned: " + str(len(references))) 
references = doc.findall('references/reference')
print("11th result concept id: " + str(references[10].find('id').text)) 

We have retrieved the first 20 results instead of the default 10.
Now let's try getting the next 20 results

    GET https://cmr.earthdata.nasa.gov/search/collections?page_size=20&page_num=2

In [ ]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?page_size=20&page_num=2")
doc = et.fromstring(response.text)
references = doc.findall('references/reference')
print("11th result concept id: " + str(references[10].find('id').text)) 

Notice that the 11th result is different because we are looking at a different page

## Getting metadata by reference
Let's get the metadata for the 11th result using the reference

In [ ]:
print("11th result reference: " + str(references[10].find('location').text)) 

response = requests.get(str(references[10].find('location').text))
response_as_dom = xml.dom.minidom.parseString(response.text)
xml_reference_response = response_as_dom.toprettyxml()
print(xml_reference_response)

## Getting metadata in the format you want
The above is the native format of the metadata. The format the data was ingested in. CMR can give us this metadata in other formats.

We can specify our desired format by file extension or an 'Accept' header in our request.
Here is a list of populat formats. For an exhaustive list see [extensions](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#extensions)

| Format  | Extension | Accept Header                               |
|---------|-----------|---------------------------------------------|
| native  | N/A       | "application/metadata+xml"                  |
| html    | .html     | "text/html"                                 |
| json    | .umm_json | "application/vnd.nasa.cmr.umm_results+json" |
| echo10  | .echo10   | "application/echo10+xml"                    |
| iso     | .iso      | "application/iso19115+xml"                  |

Let's try using a file extension to get the collection metadata in ECHO10 format

In [ ]:
response = requests.get(str(references[10].find('location').text) + ".echo10")
response_as_dom = xml.dom.minidom.parseString(response.text)
xml_reference_response = response_as_dom.toprettyxml()
print(xml_reference_response)

Let's do the same thing but using a header to specify the desired format

In [ ]:
headers = {'Accept': 'application/echo10+xml'}
response = requests.get(str(references[10].find('location').text), headers=headers)
response_as_dom = xml.dom.minidom.parseString(response.text)
xml_reference_response = response_as_dom.toprettyxml()
print(xml_reference_response)

Let's try it in json format

In [ ]:
import json
import jq

response = requests.get(str(references[10].find('location').text+ ".umm_json"))
doc = json.loads(response.text)
print(json.dumps(doc, indent=2))

We can parse the JSON response programmatically. What is the short name of this collection?

In [ ]:
print("Short name: " + jq.compile(".ShortName").input(doc).first())

## Getting metadata directly
You can also get your metadata in a single request by specifiying a format in the initial search request. Rather than getting a list of metadata locations, you will get a list of metadata records. Let's try this using UMM-JSON;

    GET https://cmr.earthdata.nasa.gov/search/collections.umm_json

In [ ]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections.umm_json")
doc = json.loads(response.text)
print(json.dumps(doc, indent=2))

You can see that we now have the actual metadata for each collection, rather than a reference to it.
Let's look at the first record;

In [ ]:
print(print(json.dumps(jq.compile(".items[0]").input(doc).first(), indent=2)))

## Filtering metadata by search constraints

You can filter your results using HTTP query parameters. Here is a simple example

In [ ]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?short_name=MOD02QKM")
print("Number of collections with this shortname: " + response.headers['CMR-Hits'])

## Parameter Options
The behavior of search with respect to each parameter can be modified using the options parameter. The options parameter takes the following form:

`options[parameter][option_key]=value`

where `parameter` is the URL parameter whose behavior is to be affected, value is either `true` or `false`, and `option_key` is one of the following:

### Ignoring case
`ignore_case` if set to true, the search will be case insensitive and if set to false, the search will be case sensitive. Defaults to true.

In [ ]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?short_name=mod02qkm&options[short_name][ignore_case]=false")
print("Number of collections with this shortname: " + response.headers['CMR-Hits'])

### Pattern matching
`pattern` if set to true, the search will treat the value provided for the parameter as a pattern with wild-cards, in which '*' matches zero or more characters and '?' matches any single character. For example, `platform[]=AB?D*&options[platform][pattern]=true` would match 'ABAD123', 'ABCD12', 'ABeD', etc. Defaults to false. 
NOTE: Patterns with leading wildcards, like "*ODIS", are computationally expensive. The CMR limits the maximum number of patterns with a leading wildcard per query to 5.

In [ ]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?short_name=D*&options[short_name][pattern]=true")
print("Number of collections with this shortname: " + response.headers['CMR-Hits'])

### AND'ing
`and` if set to true and if multiple values are listed for the param, the concepts must have ALL of these values in order to match. The default is false which means concepts with ANY of the values match. This option only applies to fields which may be multivalued; these are documented here.

In [ ]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?short_name=MOD02QKM&short_name=MOD02HKM&options[short_name][and]=true")
print("Number of collections with this shortname: " + response.headers['CMR-Hits'])